In [1]:
from bs4 import BeautifulSoup
import requests

In [9]:
url = 'https://www.espncricinfo.com/cricketers/team/afghanistan-40/alpha-a'
base_url = 'https://www.espncricinfo.com'

In [14]:
def get_content(url):
    response = requests.get(url)
    if response.status_code ==200:
        content = response.content
    else:
        content =  "Error finding page"
    return content


In [23]:
# Finds the alphabetical links of all players in a team

def find_team_aplhas(team_url):
    content = get_content(team_url)
    team_alphas = []
    if content == "Error finding page":
        return content
    soup = BeautifulSoup(content,'html.parser')
    alpha_bar = soup.find_all('div',{'class':'ds-px-4'})
    for elements in alpha_bar:
        a_tags = elements.find_all('a',{'class':'ds-h-10'})
        for tag in a_tags:
            if '/alpha-' in tag['href']:
                team_alphas.append(tag['href'])
    team_alphas = [base_url+ link for link in team_alphas]
    return team_alphas
    
    

In [16]:
# Finds all the links to player data from alphabetical links

def find_players_by_alpha(url):
    content = get_content(url)
    if content == "Error finding page":
        return content
    soup = BeautifulSoup(content, 'html.parser')
    grid = soup.find_all('div',{'class':'ds-grid'})
    players_links = []
    for div in grid:
        links = div.find_all('a',{'class':'ds-flex'})
        for link in links:
            players_link.append(base_url+link['href'])
    return players_links

https://www.espncricinfo.com/cricketers/team/afghanistan-40/alpha-a
https://www.espncricinfo.com/cricketers/team/afghanistan-40/alpha-b
https://www.espncricinfo.com/cricketers/team/afghanistan-40/alpha-d
https://www.espncricinfo.com/cricketers/team/afghanistan-40/alpha-e
https://www.espncricinfo.com/cricketers/team/afghanistan-40/alpha-f
https://www.espncricinfo.com/cricketers/team/afghanistan-40/alpha-g
https://www.espncricinfo.com/cricketers/team/afghanistan-40/alpha-h
https://www.espncricinfo.com/cricketers/team/afghanistan-40/alpha-i
https://www.espncricinfo.com/cricketers/team/afghanistan-40/alpha-j
https://www.espncricinfo.com/cricketers/team/afghanistan-40/alpha-k
https://www.espncricinfo.com/cricketers/team/afghanistan-40/alpha-l
https://www.espncricinfo.com/cricketers/team/afghanistan-40/alpha-m
https://www.espncricinfo.com/cricketers/team/afghanistan-40/alpha-n
https://www.espncricinfo.com/cricketers/team/afghanistan-40/alpha-o
https://www.espncricinfo.com/cricketers/team/afg